In [235]:
import re
import os
from PyPDF2 import PdfReader
import google.generativeai as genai

# ✅ Gemini API の設定
genai.configure(api_key=os.getenv("GEMINI_API_KEY"))

def extract_text_from_pdf(pdf_path, use_summary=True):
    """PDFからマーケット情報を抽出し、クリーニング後にGeminiで要約"""
    reader = PdfReader(pdf_path)
    raw_text = "\n".join([page.extract_text() for page in reader.pages if page.extract_text()])
    
    # ✅ クリーニング処理
    clean_text = clean_pdf_text(raw_text)
    
    if use_summary:
        summary = summarize_with_gemini(clean_text)
        return summary  # 要約のみ返す場合
        # return clean_text + "\n\n--- 要約 ---\n" + summary  # 要約を元のテキストと併せて返す場合

    return clean_text  # 要約を使わない場合はクリーンテキストのみ

def clean_pdf_text(text):
    """抽出したテキストを整形して不要な部分を削除"""

    # ✅ ページ番号やURLの削除
    text = re.sub(r"\s*[-_]+\s*\d+/\d+\s*[-_]+", "", text)  # "1/3", "2/3" などのページ表記
    text = re.sub(r"https?://\S+", "", text)  # URLを削除

    # ✅ 免責事項・注意書きを削除
    exclusion_patterns = [
        r"本資料に関してご留意頂きたい事項.*$",
        r"出所）.*?$",
        r"本資料は.*?作成されたものです。",
        r"本資料は、作成時点 で.*$",
        r"本資料の内容は.*?変更されることがあります。",
        r"本資料は.*?保証するものではありません。",
        r"本資料に示す意見等は.*?限りません。",
        r"本資料中で使用している指数について.*$",
    ]
    for pattern in exclusion_patterns:
        text = re.sub(pattern, "", text, flags=re.MULTILINE)

    # ✅ 連続する空白・改行の削除
    text = re.sub(r"\n\s*\n", "\n", text)  # 空行を削除
    text = re.sub(r"[ \t]+", " ", text)  # 連続する空白を1つに
    text = text.strip()

    # ✅ 数値だけの行・不要なヘッダーの削除
    clean_lines = []
    for line in text.split("\n"):
        if re.match(r"^\s*[\d.,/%]+\s*$", line):  
            continue
        if re.search(r"^\s*(単位|注）|.*?は.*?です。)", line):  
            continue
        clean_lines.append(line)

    return "\n".join(clean_lines).strip()

def summarize_with_gemini(text):
    """Gemini に問い合わせてテキストの要約を取得"""
    prompt = f"""
    以下のテキストから観点に沿った情報に絞って詳細を教えて。
    
    {text}
    
    観点：[1. 主要金融市場の動き,2. 主要国株式の動き,3. マーケットの動き,4.注目点]
    なお、免責事項などの記載は無駄な文章なので除外して。
    """
    try:
        gemini_model = genai.GenerativeModel("gemini-1.5-flash")
        response = gemini_model.generate_content(prompt)
        if hasattr(response, "text") and response.text:
            return response.text.strip().replace("\n","").replace("*","")
        else:
            print(f"⚠️ 要約取得失敗: {response}")
            return "要約取得に失敗しました。"
    except Exception as e:
        print(f"❌ Gemini 要約エラー: {str(e)}")
        return "要約取得中にエラーが発生しました。"


In [236]:
INPUT_DIR = "input/data-rag/"
extract_text_from_pdf(INPUT_DIR + "daily250221.pdf")

'1. 主要金融市場の動き 主要国株式:  日経平均株価は38,678.04円（前日比-486.57円）、TOPIXは2,734.60（前日比-32.65）。米国株はNYダウが44,176.65ドル（前日比-450.94ドル）、S&P500が6,117.52（前日比-26.63）、ナスダック総合指数が19,962.36（前日比-93.89）と下落。その他、ドイツDAX、英国FTSE100、豪州S&P/ASX200、中国上海総合指数、香港ハンセン指数、インドS&P BSE SENSEX指数なども下落。ブラジルボベスパ指数は上昇。MSCI WORLDは3,893.65ドル（前日比-13.17ドル）、MSCI EMは1,132.45ドル（前日比-4.42ドル）。 主要国債:  日本10年国債利回りは、2025年2月17日時点での数値は記載なし。米国、ドイツ、オーストラリアの10年国債利回りの数値は記載されているが、具体的な数値は提示されていません。グラフで確認する必要がある。 主要通貨:  米ドル/円は149.64円（前日比▲1.21円）、ユーロ/円は157.16円（前日比▲0.46円）。その他の主要通貨（英ポンド、カナダドル、豪ドル、NZドル、シンガポールドル、中国人民元、インド ルピー、インドネシアルピア、メキシコペソ、ブラジルレアル、トルコリラ、ロシアルーブル）に対円の為替レートと前日比の変動率が示されている。 商品: WTI原油先物は72.57ドル（前日比+0.32ドル）、COMEX金先物は2,940.00ドル（前日比+20.60ドル）。2. 主要国株式の動き上記「主要金融市場の動き」の主要国株式のセクションを参照。3. マーケットの動き 日本: 円高進行や日銀追加利上げ観測を嫌気し、日本株は軟調。1月消費者物価の上振れリスクへの警戒感も。 米国: ウォルマートの業績見通し予想下回ることで消費の先行き不安を意識し、米国株は軟調。スタグフレーションリスクも指摘されている。 中国: 中国人民銀行は、最優遇貸出金利を予想通り据え置き。 豪州: 1月雇用統計で失業率上昇も、就業者数は予想を大幅に上回り、豪ドルは底堅い。 ユーロ圏: 2月消費者信頼感指数が2カ月連続上昇し、ユーロは対米ドルで上昇。4. 本日の注目点米欧の製造業回復の動きが一時的なものか、持続的なものかが注目

In [237]:
extract_text_from_pdf(INPUT_DIR + "250217_weekly.pdf")

'1. 主要金融市場の動き 株式市場: 2月14日時点では、日経平均株価は+0.93%、TOPIXは+0.80%、NYダウは+0.55%、S&P500は+1.47%、ナスダック総合指数は+2.58%上昇。ストックス・ヨーロッパ600は+1.78%、DAX指数は+3.33%上昇。中国上海総合指数は+1.30%上昇。MSCI WORLDは+1.72%、MSCI EMは+1.51%上昇。S&P先進国REIT指数は+0.54%、東証REIT指数は+0.26%上昇。 長期金利（10年国債利回り）: 日本は1.350%、米国は4.477%、ドイツは2.431%、フランスは3.173%、イタリアは3.522%、スペインは3.061%、英国は4.500%、カナダは3.109%、オーストラリアは4.417%。 為替相場（対円）: 米ドルは152.31円(+0.59%)、ユーロは159.83円(+2.21%)、英ポンドは191.69円(+2.09%)、カナダドルは107.44円(+1.44%)、オーストラリアドルは96.74円(+1.88%)上昇。中国人民元は20.986円(+1.13%)上昇。 商品: 原油(WTI先物)は70.74ドル(-0.37%)、金(COMEX先物)は2,883.60ドル(+0.57%)。2. 主要国株式の動き 日本: 日経平均株価は3週ぶりに反発し、+0.9%上昇。輸出関連銘柄を中心に買いが優勢で、好調な企業決算も追い風となった。 米国: S&P500は3週ぶりに上昇し、週間で+1.47%と史上最高値に近づいた。1月CPIの上振れで一時下押す場面もあったが、トランプ政権の姿勢を市場は好感。 欧州: ストックス・ヨーロッパ600指数は史上最高値を更新。堅調な企業決算や、トランプ政権の相互関税の即時発効回避、ロシア・ウクライナの停戦期待などが市場心理の改善につながった。ドイツDAX指数も大幅上昇。 中国: 上海総合指数は上昇。 英国: 10-12月期の実質GDPは予想外にプラス成長となったが、個人消費や設備投資の弱さが懸念材料。3. マーケットの動き 株式市場は、米国を中心とした主要国の雇用環境の安定、消費主導の景気回復基調を背景に堅調を維持しているものの、米利下げ期待の後退、貿易摩擦や地政学リスクへの警戒感から、悪材料に過敏になっている。 主要国製造業の回復の

In [55]:
import google.generativeai as genai
import chromadb
import os

# 環境変数からAPIキーを設定
genai.configure(api_key=os.getenv("GEMINI_API_KEY"))

# ✅ ChromaDBのコンテナへHTTP接続
client = chromadb.HttpClient(host="chromadb", port=8000)  # chromadbのサービス名を使う

# コレクションを取得・作成
collection = client.get_or_create_collection("rag_docs")

def get_gemini_embedding(text):
    """Gemini APIを使用してテキストの埋め込みを取得"""
    model = genai.GenerativeModel("embedding-001")
    response = model.embed_content(text, task_type="retrieval_document")
    return response["embedding"]

def add_document(text, source):
    """ChromaDBに埋め込みデータを登録"""
    embedding = get_gemini_embedding(text)
    collection.add(documents=[text], metadatas=[{"source": source}], embeddings=[embedding])



In [57]:
import google.generativeai as genai
import os

# 環境変数からAPIキーを設定
genai.configure(api_key=os.getenv("GEMINI_API_KEY"))

# ✅ モデル一覧をリストとして出力
models = list(genai.list_models())

# 各モデルの情報を表示
#for model in models:
#    print(model)


In [58]:
import google.generativeai as genai
import chromadb
import os
import uuid  # ✅ 一意のIDを生成するために追加

# 環境変数からAPIキーを設定
genai.configure(api_key=os.getenv("GEMINI_API_KEY"))

# ChromaDB クライアントを作成
client = chromadb.HttpClient(host="chromadb", port=8000)  # ChromaDB のサービス名を使う
collection = client.get_or_create_collection("rag_docs")

def get_gemini_embedding(text):
    """Gemini APIを使用してテキストの埋め込みを取得"""
    response = genai.embed_content(model="models/text-embedding-004", content=text)  # ✅ 修正
    return response["embedding"]

def add_document(text, source):
    """ChromaDBに埋め込みデータを登録"""
    embedding = get_gemini_embedding(text)
    doc_id = str(uuid.uuid4())  # ✅ 一意のIDを生成
    collection.add(ids=[doc_id], documents=[text], metadatas=[{"source": source}], embeddings=[embedding])
    print(f"✅ 追加完了: {source} (ID: {doc_id})")

# ✅ 動作確認テスト
test_text = "これはテスト用のドキュメントです。"
test_source = "テストデータ"
add_document(test_text, test_source)


✅ 追加完了: テストデータ (ID: e44f8cec-ec35-489b-9f7e-fd2603d68851)


In [59]:
def search_similar_documents(query_text, n_results=5):
    """ChromaDBに保存されている文書と類似するものを検索"""
    query_embedding = get_gemini_embedding(query_text)
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=n_results
    )

    for doc, metadata, score in zip(results["documents"][0], results["metadatas"][0], results["distances"][0]):
        print(f"📌 類似ドキュメント: {doc}")
        print(f"   🔹 メタデータ: {metadata}")
        print(f"   🔹 類似度スコア: {score}")
        print("-" * 50)

# ✅ 検索テスト
search_query = "テスト用の文章"
search_similar_documents(search_query)


📌 類似ドキュメント: これはテスト用のドキュメントです。
   🔹 メタデータ: {'source': 'テストデータ'}
   🔹 類似度スコア: 0.0
--------------------------------------------------
📌 類似ドキュメント: 自然言語処理はテキストデータの分析に役立ちます。
   🔹 メタデータ: {'source': '技術記事3'}
   🔹 類似度スコア: 0.0
--------------------------------------------------
📌 類似ドキュメント: 経済学では市場の動きを分析する。
   🔹 メタデータ: {'source': '経済ニュース1'}
   🔹 類似度スコア: 0.0
--------------------------------------------------
📌 類似ドキュメント: 株式市場の動向は投資家にとって重要な指標だ。
   🔹 メタデータ: {'source': '経済ニュース2'}
   🔹 類似度スコア: 0.0
--------------------------------------------------
📌 類似ドキュメント: データサイエンスとは、データを活用した分析手法のこと。
   🔹 メタデータ: {'source': '技術記事1'}
   🔹 類似度スコア: 0.0
--------------------------------------------------


In [97]:
import google.generativeai as genai
import chromadb
import os
import uuid

# 環境変数からAPIキーを設定
genai.configure(api_key=os.getenv("GEMINI_API_KEY"))

# ✅ ChromaDBのコンテナへHTTP接続
client = chromadb.HttpClient(host="chromadb", port=8000)
collection = client.get_or_create_collection("rag_docs")

def get_gemini_embedding(text, model_name="models/text-embedding-004"):
    """Gemini APIを使用してテキストの埋め込みを取得"""
    model = genai.GenerativeModel("models/text-embedding-004")
#    response = model.embed_content(text)  # ✅ embedContent の修正
    response = genai.embed_content(model="models/text-embedding-004", content=text)  # ✅ 修正
    return response["embedding"]

def add_document(text, source):
    """ChromaDBに埋め込みデータを登録"""
    embedding = get_gemini_embedding(text)
    doc_id = str(uuid.uuid4())  # 一意のIDを作成
    collection.add(documents=[text], metadatas=[{"source": source}], embeddings=[embedding], ids=[doc_id])
    print(f"✅ 追加完了: {source} (ID: {doc_id})")

# ✅ 1. データ登録テスト
test_data = [
    ("機械学習はデータサイエンスの一部です。", "技術記事1"),
    ("ディープラーニングはニューラルネットワークを活用します。", "技術記事2"),
    ("金融市場は日々変動する。", "経済ニュース1"),
    ("為替相場の変動は経済全体に影響を与えます。", "経済ニュース2"),
    ("自然言語処理はテキストデータの分析に役立ちます。", "技術記事3"),
]

print("\n📌 データ追加テスト")
for text, source in test_data:
    add_document(text, source)

# ✅ 2. 検索テスト
search_query = "データサイエンス"
print(f"\n📌 検索テスト（類似するニュースを検索）: {search_query}")

results = collection.query(
    query_embeddings=[get_gemini_embedding(search_query)],
    n_results=3
)

# ✅ 3. 検索結果の表示
print("\n📌 検索結果:")
for i, (doc, meta, score) in enumerate(zip(results["documents"][0], results["metadatas"][0], results["distances"][0])):
    print(f"🔹 {i+1}. 類似ドキュメント: {doc}")
    print(f"   🔹 メタデータ: {meta}")
    print(f"   🔹 類似度スコア: {score:.4f}\n")



📌 データ追加テスト
✅ 追加完了: 技術記事1 (ID: 9a7e8400-8eec-4642-9546-9c023db72ced)
✅ 追加完了: 技術記事2 (ID: ac445604-8fa8-4005-9802-40c2c89ecd10)
✅ 追加完了: 経済ニュース1 (ID: cf0e8a55-6ee7-43ad-8245-6bb619c792a1)
✅ 追加完了: 経済ニュース2 (ID: b256bd46-f57b-47c6-9c80-686a616ae4be)
✅ 追加完了: 技術記事3 (ID: c946ea37-a41c-4924-a03f-6f0ed53cedc6)

📌 検索テスト（類似するニュースを検索）: データサイエンス

📌 検索結果:
🔹 1. 類似ドキュメント: ディープラーニングはニューラルネットワークを活用します。
   🔹 メタデータ: {'source': '技術記事2'}
   🔹 類似度スコア: 0.0000

🔹 2. 類似ドキュメント: 為替相場の変動は経済全体に影響を与えます。
   🔹 メタデータ: {'source': '経済ニュース2'}
   🔹 類似度スコア: 0.0000

🔹 3. 類似ドキュメント: 自然言語処理はテキストデータの分析に役立ちます。
   🔹 メタデータ: {'source': '技術記事3'}
   🔹 類似度スコア: 0.0000



In [61]:
import google.generativeai as genai
import chromadb
import os
import uuid

# 環境変数からAPIキーを設定
genai.configure(api_key=os.getenv("GEMINI_API_KEY"))

# ✅ ChromaDBのコンテナへHTTP接続
client = chromadb.HttpClient(host="chromadb", port=8000)
collection = client.get_or_create_collection("rag_docs", metadata={"hnsw:space": "cosine"})

def get_gemini_embedding(text, model_name="models/embedding-001"):
    """Gemini APIを使用してテキストの埋め込みを取得"""
    model = genai.GenerativeModel(model_name)
    response = genai.embed_content(model="models/text-embedding-004", content=text)  # ✅ 修正
    return response["embedding"]

def add_document(text, source):
    """ChromaDBに埋め込みデータを登録"""
    embedding = get_gemini_embedding(text)
    doc_id = str(uuid.uuid4())  # 一意のIDを作成
    collection.add(documents=[text], metadatas=[{"source": source}], embeddings=[embedding], ids=[doc_id])
    print(f"✅ 追加完了: {source} (ID: {doc_id})")

# ✅ 1. データ登録テスト（データを増やす）
test_data = [
    ("データサイエンスとは、データを活用した分析手法のこと。", "技術記事1"),
    ("機械学習は統計と計算機科学の融合である。", "技術記事2"),
    ("ニューラルネットワークは脳の構造を模倣する。", "技術記事3"),
    ("経済学では市場の動きを分析する。", "経済ニュース1"),
    ("株式市場の動向は投資家にとって重要な指標だ。", "経済ニュース2"),
]

print("\n📌 データ追加テスト")
for text, source in test_data:
    add_document(text, source)

# ✅ 2. 検索テスト
search_query = "データサイエンス"
print(f"\n📌 検索テスト（類似するニュースを検索）: {search_query}")

results = collection.query(
    query_embeddings=[get_gemini_embedding(search_query)],
    n_results=3
)

# ✅ 3. 検索結果の表示
print("\n📌 検索結果:")
for i, (doc, meta, score) in enumerate(zip(results["documents"][0], results["metadatas"][0], results["distances"][0])):
    print(f"🔹 {i+1}. 類似ドキュメント: {doc}")
    print(f"   🔹 メタデータ: {meta}")
    print(f"   🔹 類似度スコア: {score:.4f}\n")



📌 データ追加テスト
✅ 追加完了: 技術記事1 (ID: 5a06f535-273a-4633-b0ae-f1545f23f519)
✅ 追加完了: 技術記事2 (ID: 320ea2d4-0a9a-4b10-bec3-8b109bf77195)
✅ 追加完了: 技術記事3 (ID: 9960a008-d78b-4e4c-a47f-e8b24f11b5c5)
✅ 追加完了: 経済ニュース1 (ID: 9c24699d-8674-41c3-ad6a-2d18eac1e9c0)
✅ 追加完了: 経済ニュース2 (ID: 78d6f336-4f4f-4357-b7d0-482cabc928ae)

📌 検索テスト（類似するニュースを検索）: 家事について

📌 検索結果:
🔹 1. 類似ドキュメント: これはテスト用のドキュメントです。
   🔹 メタデータ: {'source': 'テストデータ'}
   🔹 類似度スコア: 0.0000

🔹 2. 類似ドキュメント: データサイエンスとは、データを活用した分析手法のこと。
   🔹 メタデータ: {'source': '技術記事1'}
   🔹 類似度スコア: 0.0000

🔹 3. 類似ドキュメント: 株式市場の動向は投資家にとって重要な指標だ。
   🔹 メタデータ: {'source': '経済ニュース2'}
   🔹 類似度スコア: 0.0000



In [62]:
import google.generativeai as genai
import chromadb
import os
import uuid
import numpy as np

# 環境変数からAPIキーを設定
genai.configure(api_key=os.getenv("GEMINI_API_KEY"))

# ✅ ChromaDBのコンテナへHTTP接続
client = chromadb.HttpClient(host="chromadb", port=8000)
collection = client.get_or_create_collection("rag_docs", metadata={"hnsw:space": "cosine"})

def get_gemini_embedding(text, model_name="models/embedding-001"):
    """Gemini APIを使用してテキストの埋め込みを取得"""
    model = genai.GenerativeModel(model_name)
    response = genai.embed_content(model="models/text-embedding-004", content=text)  # ✅ 修正
    return response["embedding"]

def add_document(text, source):
    """ChromaDBに埋め込みデータを登録"""
    embedding = get_gemini_embedding(text)
    doc_id = str(uuid.uuid4())  # 一意のIDを作成
    collection.add(documents=[text], metadatas=[{"source": source}], embeddings=[embedding], ids=[doc_id])
    print(f"✅ 追加完了: {source} (ID: {doc_id})")

# ✅ 1. データ登録テスト（類似データを増やす）
test_data = [
    ("データサイエンスとは、データを活用した分析手法のこと。", "技術記事1"),
    ("データサイエンスは機械学習と統計の融合です。", "技術記事2"),  # 類似データ
    ("ニューラルネットワークはAIの中心技術の一つです。", "技術記事3"),
    ("データ解析の手法には回帰分析やクラスタリングがあります。", "技術記事4"),  # 類似データ
    ("経済学では市場の動きを分析する。", "経済ニュース1"),
    ("為替市場の変動は経済全体に影響を与えます。", "経済ニュース2"),
]

print("\n📌 データ追加テスト")
for text, source in test_data:
    add_document(text, source)

# ✅ 2. 検索テスト
search_query = "データ分析"
print(f"\n📌 検索テスト（類似するニュースを検索）: {search_query}")

query_embedding = get_gemini_embedding(search_query)

results = collection.query(
    query_embeddings=[query_embedding],
    n_results=3
)

# ✅ 3. 検索結果の表示
print("\n📌 検索結果:")
for i, (doc, meta, score) in enumerate(zip(results["documents"][0], results["metadatas"][0], results["distances"][0])):
    print(f"🔹 {i+1}. 類似ドキュメント: {doc}")
    print(f"   🔹 メタデータ: {meta}")
    print(f"   🔹 類似度スコア: {score:.4f}\n")

# ✅ 4. 埋め込みベクトルの分布を確認
all_embeddings = np.array([get_gemini_embedding(text) for text, _ in test_data] + [query_embedding])
mean_vector = np.mean(all_embeddings, axis=0)
std_dev_vector = np.std(all_embeddings, axis=0)

print("\n📊 埋め込みベクトルの統計情報:")
print(f"   平均値: {np.mean(mean_vector):.4f}, 標準偏差: {np.mean(std_dev_vector):.4f}")



📌 データ追加テスト
✅ 追加完了: 技術記事1 (ID: c036578b-4796-4887-a6a8-975fd6ffc75a)
✅ 追加完了: 技術記事2 (ID: c78d3fc7-f577-489c-9a83-24895a698420)
✅ 追加完了: 技術記事3 (ID: c589b008-2968-4100-9320-1c144ec921cf)
✅ 追加完了: 技術記事4 (ID: eab9530e-d05e-4b85-b5e4-2b33ad9ff3f5)
✅ 追加完了: 経済ニュース1 (ID: 1c60ed29-aa27-4d3e-975e-80af6588ddfc)
✅ 追加完了: 経済ニュース2 (ID: 539a0216-3fdd-441b-8126-6d509dc6084e)

📌 検索テスト（類似するニュースを検索）: データ分析

📌 検索結果:
🔹 1. 類似ドキュメント: これはテスト用のドキュメントです。
   🔹 メタデータ: {'source': 'テストデータ'}
   🔹 類似度スコア: 0.0000

🔹 2. 類似ドキュメント: データサイエンスとは、データを活用した分析手法のこと。
   🔹 メタデータ: {'source': '技術記事1'}
   🔹 類似度スコア: 0.0000

🔹 3. 類似ドキュメント: 株式市場の動向は投資家にとって重要な指標だ。
   🔹 メタデータ: {'source': '経済ニュース2'}
   🔹 類似度スコア: 0.0000


📊 埋め込みベクトルの統計情報:
   平均値: -0.0011, 標準偏差: 0.0080


In [63]:
!pip install pdfplumber

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [72]:
results

{'ids': [['88e2d6d5-f77d-4a0c-8663-17ef0a9e561b',
   '62db6eff-6782-40e2-a4a5-ac688cd1be55']],
 'distances': [[0.18927793204784393, 0.18927793204784393]],
 'embeddings': None,
 'metadatas': [[{'source': 'テストデータ'}, {'source': '技術記事1'}]],
 'documents': [['これはテスト用のドキュメントです。', 'データサイエンスとは、データを活用した分析手法のこと。']],
 'uris': None,
 'data': None,
 'included': ['distances', 'documents', 'metadatas']}

In [92]:
collection.peek()

InvalidCollectionException: Collection af98fc14-4183-4ee5-a30f-cb07ad5f6cee does not exist.

In [67]:
def get_gemini_embedding(text):
    """Gemini APIを使用してテキストの埋め込みを取得"""
    response = genai.embed_content(model="models/text-embedding-004", content=text)  # ✅ 768次元のモデルを指定
    return response["embedding"]

query_text = "機械学習って何？"
query_embedding = get_gemini_embedding(query_text)

results = collection.query(
    query_embeddings=[query_embedding],  # ✅ クエリを埋め込みベクトルに変換して送る
    n_results=2
)
print(results)

{'ids': [['88e2d6d5-f77d-4a0c-8663-17ef0a9e561b', '62db6eff-6782-40e2-a4a5-ac688cd1be55']], 'distances': [[0.18927793204784393, 0.18927793204784393]], 'embeddings': None, 'metadatas': [[{'source': 'テストデータ'}, {'source': '技術記事1'}]], 'documents': [['これはテスト用のドキュメントです。', 'データサイエンスとは、データを活用した分析手法のこと。']], 'uris': None, 'data': None, 'included': ['distances', 'documents', 'metadatas']}


In [40]:
import os
import pdfplumber
import chromadb
import google.generativeai as genai
import uuid

# 環境変数からAPIキーを設定
genai.configure(api_key=os.getenv("GEMINI_API_KEY"))

# ChromaDB の HTTP クライアント（Docker コンテナの chromadb に接続）
client = chromadb.HttpClient(host="chromadb", port=8000)

# ✅ コレクション作成（用途別）
collections = {
    "daily": client.get_or_create_collection("daily_docs"),
    "weekly": client.get_or_create_collection("weekly_docs"),
}

# ✅ テキストの埋め込み取得関数
def get_gemini_embedding(text):
    """Gemini APIを使用してテキストの埋め込みを取得"""
    model = genai.GenerativeModel("models/text-embedding-004")  # 適切な埋め込みモデルを選択
    response = genai.embed_content(model="models/text-embedding-004", content=text)  # ✅ 修正
    return response["embedding"]

# ✅ PDFのテキストを用途別に登録
def process_pdf(file_path, category):
    """PDFファイルを読み込んで用途別にChromaDBへ登録"""
    with pdfplumber.open(file_path) as pdf:
        full_text = "\n".join([page.extract_text() for page in pdf.pages if page.extract_text()])
    
    if not full_text.strip():
        print(f"⚠️ {file_path} はテキストを抽出できませんでした")
        return
    
    embedding = get_gemini_embedding(full_text)
    doc_id = str(uuid.uuid4())  # ユニークなIDを生成

    collections[category].add(
        ids=[doc_id],
        documents=[full_text],
        metadatas=[{"source": file_path, "category": category}],
        embeddings=[embedding]
    )
    
    print(f"✅ 登録完了: {file_path} → {category} (ID: {doc_id})")

INPUT_DIR = "input/data-rag/"
extract_text_from_pdf(INPUT_DIR + "daily250221.pdf")

extract_text_from_pdf(INPUT_DIR + "weekly_exchange.pdf")

# ✅ テスト用PDFの登録（用途に応じて変更）
pdf_files = {
    INPUT_DIR + "daily250221.pdf": "daily",
    INPUT_DIR + "weekly_exchange.pdf": "weekly",
#    "sample_news.pdf": "news",
#    "sample_other.pdf": "other",
}

for file, category in pdf_files.items():
    process_pdf(file, category)


✅ 登録完了: input/data-rag/daily250221.pdf → daily (ID: 0f0924ba-c58c-4293-a43a-b1a562ab5ade)


InvalidArgument: 400 Request payload size exceeds the limit: 10000 bytes.

In [74]:
import os
import pdfplumber
import chromadb
import google.generativeai as genai
import uuid

# 環境変数からAPIキーを設定
genai.configure(api_key=os.getenv("GEMINI_API_KEY"))

# ChromaDB の HTTP クライアント（Docker コンテナの chromadb に接続）
client = chromadb.HttpClient(host="chromadb", port=8000)

# ✅ コレクション作成
collections = {
    "daily": client.get_or_create_collection("daily_docs"),
    "weekly": client.get_or_create_collection("weekly_docs"),
    "monthly": client.get_or_create_collection("monthly_docs"),
}

# ✅ テキストの埋め込み取得関数（10,000バイト制限対応）
def get_gemini_embedding(text):
    """Gemini APIを使用してテキストの埋め込みを取得"""
    try:
        model = genai.GenerativeModel("models/text-embedding-004")  
        response = genai.embed_content(model="models/text-embedding-004", content=text)  # ✅ 修正
        return response["embedding"]
    except Exception as e:
        print(f"❌ 埋め込みエラー: {str(e)}")
        return None

# ✅ バイト数でテキストを分割する関数
def split_text_by_bytes(text, max_bytes=8000):
    """UTF-8エンコーディング時のバイト数に基づいてテキストを分割"""
    chunks = []
    current_chunk = ""

    for sentence in text.split("\n"):
        if len((current_chunk + sentence).encode("utf-8")) > max_bytes:
            chunks.append(current_chunk)
            current_chunk = sentence  # 新しいチャンク開始
        else:
            current_chunk += "\n" + sentence

    if current_chunk:
        chunks.append(current_chunk)

    return chunks

# ✅ PDFのテキストを用途別に登録（チャンク分割対応）
def process_pdf(file_path, category):
    """PDFファイルを読み込んで用途別にChromaDBへ登録（チャンク分割対応）"""
    if category not in collections:
        print(f"⚠️ 未知のカテゴリ '{category}' を 'other' に変更")
        category = "other"

    with pdfplumber.open(file_path) as pdf:
        full_text = "\n".join([page.extract_text() for page in pdf.pages if page.extract_text()])
    
    if not full_text.strip():
        print(f"⚠️ {file_path} はテキストを抽出できませんでした")
        return
    
    # ✅ 8000バイト未満のチャンクに分割
    text_chunks = split_text_by_bytes(full_text, max_bytes=8000)

    for idx, chunk in enumerate(text_chunks):
        embedding = get_gemini_embedding(chunk)
        if embedding is None:
            print(f"⚠️ 埋め込み失敗: チャンク {idx+1}/{len(text_chunks)} (ファイル: {file_path})")
            continue

        doc_id = str(uuid.uuid4())  # ユニークなIDを生成

        try:
            collections[category].add(
                ids=[doc_id],
                documents=[chunk],
                metadatas=[{"source": file_path, "category": category, "chunk_index": idx}],
                embeddings=[embedding]
            )
            print(f"✅ チャンク {idx+1}/{len(text_chunks)} 登録完了: {file_path} → {category} (ID: {doc_id})")

        except Exception as e:
            print(f"❌ ChromaDB 登録エラー: {str(e)}")

# ✅ テスト用PDFの登録
INPUT_DIR = "input/data-rag/"
pdf_files = {
    INPUT_DIR + "daily250221.pdf": "daily",
    INPUT_DIR + "daily250220.pdf": "daily",
    INPUT_DIR + "daily250219.pdf": "daily",
    INPUT_DIR + "daily250218.pdf": "daily",
    INPUT_DIR + "250217_weekly.pdf": "weekly",
    INPUT_DIR + "250210_weekly.pdf": "weekly",
    INPUT_DIR + "250203_weekly.pdf": "weekly",
    INPUT_DIR + "monthly_2502.pdf": "monthly",
}

for file, category in pdf_files.items():
    process_pdf(file, category)


✅ チャンク 1/2 登録完了: input/data-rag/daily250221.pdf → daily (ID: dc09b480-5b88-4bb7-8614-e672be0b6f7c)
✅ チャンク 2/2 登録完了: input/data-rag/daily250221.pdf → daily (ID: cdf406a4-a5e9-4d7a-bc10-8fdd3ad0a30b)
✅ チャンク 1/2 登録完了: input/data-rag/daily250220.pdf → daily (ID: 3bd06dd7-831d-404f-b67e-c9a97d820b54)
✅ チャンク 2/2 登録完了: input/data-rag/daily250220.pdf → daily (ID: da09d035-3443-49c1-aa88-76f33ff9ebd6)
✅ チャンク 1/2 登録完了: input/data-rag/daily250219.pdf → daily (ID: 3af5b42d-f02e-4641-9293-66215a26a2b4)
✅ チャンク 2/2 登録完了: input/data-rag/daily250219.pdf → daily (ID: a50b469d-215e-4d7e-a066-63113f49bece)
✅ チャンク 1/2 登録完了: input/data-rag/daily250218.pdf → daily (ID: c2fd902b-8567-4270-acb6-e6334191f11c)
✅ チャンク 2/2 登録完了: input/data-rag/daily250218.pdf → daily (ID: 28f79faf-4920-4c7f-b025-b23ad3e379b3)


FileNotFoundError: [Errno 2] No such file or directory: 'input/data-rag/daily250217.pdf'

In [75]:
import chromadb

# ChromaDB クライアント（Docker の chromadb に接続）
client = chromadb.HttpClient(host="chromadb", port=8000)

# ✅ コレクション一覧を取得（修正後）
collections = client.list_collections()
print("📌 登録されているコレクション:")
for col in collections:
    print(f"- {col}")  # `.name` は不要

📌 登録されているコレクション:
- weekly_docs
- daily_docs
- monthly_docs


In [76]:
# ✅ コレクション名を指定して取得
collection_name = "daily_docs"  # 確認したいコレクション名
collection = client.get_collection(collection_name)

# ✅ 登録されているドキュメントを取得（最初の5件）
docs = collection.get(include=["documents", "metadatas", "embeddings"], limit=5)

print("\n📌 コレクションの内容:")
for i, doc in enumerate(docs["documents"]):
    print(f"\n🔹 ID: {docs['ids'][i]}")
    print(f"📄 テキスト: {doc[:200]}...")  # 長すぎる場合は200文字に制限
    print(f"📝 メタデータ: {docs['metadatas'][i]}")



📌 コレクションの内容:

🔹 ID: dc09b480-5b88-4bb7-8614-e672be0b6f7c
📄 テキスト: 
D 情報提供資料
投資環境デイリー
2025年2月21日号
経済調査室
s
u 米国株下落、米個人消費の先行き不安を意識
c
o
F
主要金融市場の動き 主要国株式の動き
株式 （単位：ポイント） 2月20日 2月19日 前日差 48,000 （日経平均株価:円、NYダウ:米ドル） （DAX®:ポイント） 25,000
日本 日経平均株価 （円） 38,678.04 39,164.61 -486...
📝 メタデータ: {'category': 'daily', 'chunk_index': 0, 'source': 'input/data-rag/daily250221.pdf'}

🔹 ID: cdf406a4-a5e9-4d7a-bc10-8fdd3ad0a30b
📄 テキスト: ドイツ「DAX®」：本指数は、情報提供を目的としており、売買等を推奨するものではありません。
FTSEInternationalLimited(“FTSE”)©FTSE。“FTSE®”はロンドン証券取引所グループ会社の登録商標であり、FTSEInternationalLimitedは許可を得
て使用しています。FTSE指数、FTSE格付け、またはその両方におけるすべての権利は、FTSE、そのライセ...
📝 メタデータ: {'category': 'daily', 'chunk_index': 1, 'source': 'input/data-rag/daily250221.pdf'}

🔹 ID: 3bd06dd7-831d-404f-b67e-c9a97d820b54
📄 テキスト: 
D 情報提供資料
投資環境デイリー
2025年2月20日号
経済調査室
s
u 日欧株は下落、トランプ米政権の関税政策に警戒感
c
o
F
主要金融市場の動き 主要国株式の動き
株式 （単位：ポイント） 2月19日 2月18日 前日差 46,000 （日経平均株価:円、NYダウ:米ドル） （DAX®:ポイント） 24,000
日本 日経平均株価 （円） 39,164.61 39,270.40 -...
📝 メタデータ: {'category': 'daily', '

In [77]:
print(client.list_collections())

['weekly_docs', 'daily_docs', 'monthly_docs']


In [78]:
import chromadb

# ChromaDB クライアント（Docker の chromadb に接続）
client = chromadb.HttpClient(host="chromadb", port=8000)

def retrieve_relevant_macro_info(query: str, collection_name="daily_docs", top_k=3):
    """ChromaDBから最新のマクロ情報を取得"""
    collection = client.get_collection(collection_name)
    
    # クエリに基づいて類似情報を検索
    results = collection.query(
        query_texts=[query], 
        n_results=top_k, 
        include=["documents", "metadatas", "distances"]
    )

    # 検索結果のフォーマット
    retrieved_docs = []
    for i in range(len(results["documents"])):
        retrieved_docs.append(f"🔹 {results['documents'][i][:200]}...")  # 200文字に制限
        retrieved_docs.append(f"📝 {results['metadatas'][i]}")

    return "\n".join(retrieved_docs) if retrieved_docs else "🔍 関連情報なし"


In [79]:
import chromadb

# ChromaDB の HTTP クライアント（Docker の chromadb に接続）
client = chromadb.HttpClient(host="chromadb", port=8000)

# 取得するコレクション名（用途に応じて変更）
collection_name = "daily_docs"

# ChromaDB からコレクションを取得
collection = client.get_collection(collection_name)

# 取得成功メッセージ
print(f"✅ ChromaDB コレクション '{collection_name}' に接続しました！")

✅ ChromaDB コレクション 'daily_docs' に接続しました！


In [82]:
def get_gemini_embedding(text):
    """Gemini APIを使用してテキストの埋め込みを取得"""
    response = genai.embed_content(model="models/text-embedding-004", content=text)  # ✅ 768次元のモデルを指定
    return response["embedding"]

def retrieve_relevant_info(query: str, top_k=3):
    """ChromaDB から類似情報を取得"""
    query_embedding = get_gemini_embedding(query)  # ✅ クエリをベクトル化

    results = collection.query(
        query_embeddings=[query_embedding],  # ✅ クエリを埋め込みベクトルに変換して送る
        n_results=top_k,  # 取得する類似データ数
        include=["documents", "metadatas", "distances"]  # 結果に含める情報
    )

    # 取得結果を整形
    if not results["documents"]:
        return "🔍 関連情報なし"

    retrieved_docs = []
    for i in range(len(results["documents"][0])):  # top_k の件数分ループ
        retrieved_docs.append(f"🔹 類似情報 {i+1}: {results['documents'][0][i][:200]}...")
        retrieved_docs.append(f"📝 メタデータ: {results['metadatas'][0][i]}")
        retrieved_docs.append(f"🔢 類似度スコア: {results['distances'][0][i]:.4f}")
        retrieved_docs.append("-" * 50)

    return "\n".join(retrieved_docs)

query_text = "最近の金融市場動向"
retrieved_info = retrieve_relevant_info(query_text, top_k=3)

print("\n=== 🔥 ChromaDB RAG 検索結果 ===")
print(retrieved_info)



=== 🔥 ChromaDB RAG 検索結果 ===
🔹 類似情報 1: 
D 情報提供資料
投資環境デイリー
2025年2月18日号
経済調査室
s
u 日本の好調な経済成長を受けて国内金利上昇、円高の動き
c
o
F
主要金融市場の動き 主要国株式の動き
株式 （単位：ポイント） 2月17日 2月14日 前日差 （日経平均株価:円、NYダウ:米ドル） （DAX®:ポイント）
46,000 24,000
日本 日経平均株価 （円） 39,174.25 39,149.4...
📝 メタデータ: {'category': 'daily', 'chunk_index': 0, 'source': 'input/data-rag/daily250218.pdf'}
🔢 類似度スコア: 0.9981
--------------------------------------------------
🔹 類似情報 2: 
D 情報提供資料
投資環境デイリー
2025年2月20日号
経済調査室
s
u 日欧株は下落、トランプ米政権の関税政策に警戒感
c
o
F
主要金融市場の動き 主要国株式の動き
株式 （単位：ポイント） 2月19日 2月18日 前日差 46,000 （日経平均株価:円、NYダウ:米ドル） （DAX®:ポイント） 24,000
日本 日経平均株価 （円） 39,164.61 39,270.40 -...
📝 メタデータ: {'category': 'daily', 'chunk_index': 0, 'source': 'input/data-rag/daily250220.pdf'}
🔢 類似度スコア: 0.9997
--------------------------------------------------
🔹 類似情報 3: 
D 情報提供資料
投資環境デイリー
2025年2月19日号
経済調査室
s
u 株式市場は続伸、ウクライナｰロシアの戦争終結への期待高まる
c
o
F
主要金融市場の動き 主要国株式の動き
株式 （単位：ポイント） 2月18日 2月17日 前日差 （日経平均株価:円、NYダウ:米ドル） （DAX®:ポイント）
46,000 24,000
日本 日経平均株価 （円） 39,270.40 39,17...
📝 メタデータ

In [91]:
# コレクションの削除
client.delete_collection("daily_docs")  # コレクション名を指定
client.delete_collection("weekly_docs")  # コレクション名を指定
client.delete_collection("monthly_docs")  # コレクション名を指定

In [85]:
!pip install langchain

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 3.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 4.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 414.1/414.1 kB 9.0 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 333.0/333.0 kB 8.8 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.2/14.2 MB 5.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.4/276.4 kB 2.8 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.1/131.1 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.4/232.4 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 340.6/340.6 kB 4.9 MB/s eta 0:00:0

In [151]:
import os
import pdfplumber
import chromadb
import google.generativeai as genai
import uuid
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 環境変数からAPIキーを設定
genai.configure(api_key=os.getenv("GEMINI_API_KEY"))

# ChromaDB の HTTP クライアント（Docker コンテナの chromadb に接続）
client = chromadb.HttpClient(host="chromadb", port=8000)

# ✅ コレクション作成
collections = {
    "daily": client.get_or_create_collection("daily_docs"),
    "weekly": client.get_or_create_collection("weekly_docs"),
    "monthly": client.get_or_create_collection("monthly_docs"),
}

# ✅ 適切な埋め込みモデルを利用
EMBEDDING_MODEL = "models/text-embedding-004"  # 高精度な768次元埋め込み

def get_gemini_embedding(text):
    """Gemini APIを使用してテキストの埋め込みを取得"""
    try:
        response = genai.embed_content(model=EMBEDDING_MODEL, content=text)
        if "embedding" in response:
            return response["embedding"]
        else:
            print(f"⚠️ 埋め込み取得失敗（レスポンス不正）: {response}")
            return None
    except Exception as e:
        print(f"❌ 埋め込みエラー: {str(e)}")
        return None

# ✅ チャンク化（トークンベースで分割 & オーバーラップ設定）
def split_text(text, chunk_size=300, overlap=50):
    """テキストを適切なチャンクサイズで分割"""
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, chunk_overlap=overlap, length_function=len
    )
    return text_splitter.split_text(text)

# ✅ PDFのテキストを用途別に登録（チャンク分割対応）
def process_pdf(file_path, category):
    """PDFファイルを読み込んで用途別にChromaDBへ登録（チャンク分割対応）"""
    if category not in collections:
        print(f"⚠️ 未知のカテゴリ '{category}' を 'other' に変更")
        category = "other"

    with pdfplumber.open(file_path) as pdf:
        full_text = "\n".join([page.extract_text() for page in pdf.pages if page.extract_text()])
    
    if not full_text.strip():
        print(f"⚠️ {file_path} はテキストを抽出できませんでした")
        return
    
    # ✅ トークンベースで分割
    text_chunks = split_text(full_text)

    for idx, chunk in enumerate(text_chunks):
        embedding = get_gemini_embedding(chunk)
        if embedding is None:
            print(f"⚠️ 埋め込み失敗: チャンク {idx+1}/{len(text_chunks)} (ファイル: {file_path})")
            continue

        doc_id = str(uuid.uuid4())  # ユニークなIDを生成

        try:
            collections[category].add(
                ids=[doc_id],
                documents=[chunk],
                metadatas=[{"source": file_path, "category": category, "chunk_index": idx}],
                embeddings=[embedding]
            )
            print(f"✅ チャンク {idx+1}/{len(text_chunks)} 登録完了: {file_path} → {category} (ID: {doc_id})")

        except Exception as e:
            print(f"❌ ChromaDB 登録エラー: {str(e)}")

# ✅ ChromaDB 内のデータを確認
def check_chromadb_data():
    """ChromaDB にデータが正しく登録されているか確認"""
    for category, collection in collections.items():
        try:
            data = collection.peek()
            print(f"📌 {category} コレクションのデータ:")
            print(data)
        except Exception as e:
            print(f"⚠️ コレクション {category} のデータ確認エラー: {str(e)}")

# ✅ クエリのリライト（LLMを利用）
def rewrite_query(query):
    """LLM を使ってクエリを拡張"""
    prompt = f"""
    ユーザーのクエリをより具体的に拡張してください。
    
    クエリ: {query}
    
    出力例: 2025年2月の金融市場動向と米国株式市場の関係
    """
    try:
        gemini_model = genai.GenerativeModel("gemini-1.5-flash")
        response = gemini_model.generate_content(prompt)
        if hasattr(response, "text") and response.text:
            print(f"🟢 クエリ拡張成功: {response.text}")
            return response.text.strip()
        else:
            print(f"⚠️ クエリ拡張失敗（レスポンス不正）: {response}")
            return query
    except Exception as e:
        print(f"❌ クエリ拡張エラー: {str(e)}")
        return query

# ✅ ChromaDB に問い合わせ（類似度フィルタ適用）
def retrieve_relevant_info(query: str, top_k=3, min_score=0.1):
    """ChromaDB から類似情報を取得（類似度閾値なし）"""
    expanded_query = rewrite_query(query)  # クエリ拡張
    print(f"🟢 クエリ拡張後: {expanded_query}")

    query_embedding = get_gemini_embedding(expanded_query)  # ✅ クエリをベクトル化
    if query_embedding is None:
        print("❌ クエリ埋め込みの取得に失敗しました")
        return "🔍 クエリの埋め込みに失敗しました。"

    # ✅ クエリ埋め込みの次元確認
    print(f"✅ クエリの埋め込みベクトルの次元: {len(query_embedding)}")

    # ✅ ChromaDB に格納されているデータの埋め込み次元確認
    peek_data = collections["daily"].peek()
    if "embeddings" in peek_data and len(peek_data["embeddings"]) > 0:
        print(f"✅ 格納済みデータの埋め込みベクトルの次元: {len(peek_data['embeddings'][0])}")
    
    try:
        results = collections["daily"].query(
            query_embeddings=[query_embedding],
            n_results=top_k,
            include=["documents", "metadatas", "distances"]
        )
    except Exception as e:
        print(f"❌ クエリ実行エラー: {str(e)}")
        return "🔍 クエリの実行に失敗しました。"

    if not results.get("documents"):
        return "🔍 関連情報なし"

    retrieved_docs = []
    for i in range(len(results["documents"][0])):
        score = results["distances"][0][i]
        if score < min_score:
            continue  # スコアが低すぎる場合は無視

        retrieved_docs.append(f"🔹 類似情報 {i+1}: {results['documents'][0][i][:200]}...")
        retrieved_docs.append(f"📝 メタデータ: {results['metadatas'][0][i]}")
        retrieved_docs.append(f"🔢 類似度スコア: {score:.4f}")
        retrieved_docs.append("-" * 50)

    return "\n".join(retrieved_docs) if retrieved_docs else "🔍 適切な情報が見つかりませんでした。"
    for i in range(len(results["documents"][0])):
        retrieved_docs.append(f"🔹 類似情報 {i+1}: {results['documents'][0][i][:200]}...")
        retrieved_docs.append(f"📝 メタデータ: {results['metadatas'][0][i]}")
        retrieved_docs.append(f"🔢 類似度スコア: {results['distances'][0][i]:.4f}")
        retrieved_docs.append("-" * 50)

    return "\n".join(retrieved_docs) if retrieved_docs else "🔍 適切な情報が見つかりませんでした。"

# ✅ LLM に検索結果を要約させる
def summarize_retrieved_info(query: str, top_k=3):
    """RAG 検索結果を LLM で要約"""
    retrieved_docs = retrieve_relevant_info(query, top_k)
    
    if "🔍 適切な情報が見つかりませんでした。" in retrieved_docs:
        print("❌ 要約不可: 適切な検索結果がありません。")
        return "🔍 要約できる情報が見つかりませんでした。"

    summary_prompt = f"""
    あなたは金融市場の専門家です。
    以下の情報を要約し、最近の金融市場の動向について詳しく解説してください。

    ### 🔍 検索結果:
    {retrieved_docs}

    ---
    
    要約:
    """
    try:
        gemini_model = genai.GenerativeModel("gemini-1.5-flash")
        response = gemini_model.generate_content(summary_prompt)
        
        if hasattr(response, "text") and response.text:
            print(f"📝 要約成功: {response.text.strip()}")
            return response.text.strip()
        else:
            print(f"⚠️ 要約失敗（レスポンス不正）: {response}")
            return "🔍 要約生成に失敗しました。"

    except Exception as e:
        print(f"❌ 要約エラー: {str(e)}")
        return "🔍 要約の生成に失敗しました。"

In [ ]:
# ✅ テスト用PDFの登録
INPUT_DIR = "input/data-rag/"
pdf_files = {
    INPUT_DIR + "daily250221.pdf": "daily",
    INPUT_DIR + "daily250220.pdf": "daily",
    INPUT_DIR + "daily250219.pdf": "daily",
    INPUT_DIR + "daily250218.pdf": "daily",
    INPUT_DIR + "250217_weekly.pdf": "weekly",
    INPUT_DIR + "250210_weekly.pdf": "weekly",
    INPUT_DIR + "250203_weekly.pdf": "weekly",
    INPUT_DIR + "monthly_2502.pdf": "monthly",
}

for file, category in pdf_files.items():
    process_pdf(file, category)

In [155]:
# ✅ 検証用コード
if __name__ == "__main__":
    check_chromadb_data()  # ChromaDB にデータが登録されているか確認

    # クエリの実行と LLM での要約
    query_text = "マーケットの動きとして日本動向を教えて"
    summary_info = summarize_retrieved_info(query_text, top_k=3)

    print("\n=== 🔥 LLM による要約結果 ===")
    print(summary_info)


📌 daily コレクションのデータ:
{'ids': ['e11eb683-06b3-43ce-9335-00265a6bbcce', 'ea1fbf0e-5e74-4c63-a394-6f71c359307d', 'effac6bc-a97e-4301-9ecc-0a2b146d4028', '2286e071-66d7-4d95-b235-ca62335673e9', '1f42c9a3-4219-48f5-9105-18bc126f0458', '012c83bf-b44d-4b2f-bf4a-bb6508760dbd', 'ca9cdedd-bb77-4e46-b286-04acceb57b52', '87e8a556-bb36-4ed7-bc13-fdb0efab89c5', '21ab6f28-6791-49d8-86d3-a6b67f8861d2', '843de30d-9ae5-458f-93ab-7e6d06ec27c0'], 'embeddings': array([[ 0.0447529 ,  0.06725755, -0.035169  , ..., -0.04462361,
         0.05318881, -0.05041842],
       [ 0.04112265,  0.03121432,  0.024434  , ..., -0.05765102,
         0.0257796 , -0.04316479],
       [ 0.02719103,  0.04258162, -0.04399061, ..., -0.06919848,
         0.05181579, -0.04099518],
       ...,
       [ 0.00447275,  0.04896156, -0.05324043, ...,  0.00534078,
         0.07214562, -0.00303894],
       [-0.02352262,  0.03660674, -0.00744899, ...,  0.00512246,
         0.07658856, -0.03041998],
       [ 0.03006796,  0.07615107, -0.0325649

In [135]:
#原因調査

In [147]:
query_text = "最近の金融市場動向"
query_embedding = get_gemini_embedding(query_text)

peek_data = collections["daily"].peek()
stored_embedding = peek_data["embeddings"][0]  # 最初のデータ

similarity = sum(q * s for q, s in zip(query_embedding, stored_embedding))  # 内積計算
print(f"🔍 クエリと最初のデータの類似度: {similarity}")


🔍 クエリと最初のデータの類似度: 0.47075930205331534


In [141]:
peek_data = collections["daily"].peek()
print(f"✅ 格納済みデータの埋め込みベクトルの次元: {len(peek_data['embeddings'][0])}")

✅ 格納済みデータの埋め込みベクトルの次元: 768


In [143]:
query_text = "最近の金融市場動向"
expanded_query = rewrite_query(query_text)
print(f"🟢 クエリ拡張後: {expanded_query}")


🟢 クエリ拡張成功: いくつかの拡張例を挙げます。元のクエリ「最近の金融市場動向」は非常に広いため、より具体的な情報を引き出すには、時間軸、市場の種類、影響要因など、様々な要素を考慮する必要があります。

**時間軸を絞り込んだ例:**

* **2024年第4四半期のグローバル金融市場動向とインフレ率の関係**
* **過去3ヶ月の主要中央銀行の金融政策と債券市場への影響**
* **今年度の日本の金融市場動向と円安の関連性**

**市場の種類を指定した例:**

* **最近のテクノロジー株市場の動向とAI関連企業の株価変動**
* **最近の原油価格の動向とエネルギーセクターへの影響**
* **最近の暗号通貨市場の動向と規制の影響**

**影響要因を指定した例:**

* **最近の地政学的リスクが金融市場に与える影響と今後の展望**
* **最近のサプライチェーン問題が金融市場動向に及ぼす影響**
* **最近の金利上昇が不動産市場に与える影響**

**より複合的な例:**

* **2024年後半における米国と中国の金融市場の相互作用と貿易摩擦の影響**
* **最近のインフレ抑制策と主要国の金融政策による株式市場への影響分析**
* **持続可能な投資への関心の高まりが最近の金融市場動向に与えている影響**


このように、元のクエリを拡張するには、**「いつ」「どこで」「何が」「なぜ」**という点を具体的に記述することが重要です。  より具体的なクエリを作成することで、より正確で有益な情報を得ることができます。

🟢 クエリ拡張後: いくつかの拡張例を挙げます。元のクエリ「最近の金融市場動向」は非常に広いため、より具体的な情報を引き出すには、時間軸、市場の種類、影響要因など、様々な要素を考慮する必要があります。

**時間軸を絞り込んだ例:**

* **2024年第4四半期のグローバル金融市場動向とインフレ率の関係**
* **過去3ヶ月の主要中央銀行の金融政策と債券市場への影響**
* **今年度の日本の金融市場動向と円安の関連性**

**市場の種類を指定した例:**

* **最近のテクノロジー株市場の動向とAI関連企業の株価変動**
* **最近の原油価格の動向とエネルギーセクターへの影響**
* **最近の暗号通貨市場の動向と規制の

In [139]:
query_text = "最近の金融市場動向"
rewrite_query(query_text)

🟢 クエリ拡張成功: いくつかの具体的に拡張したクエリ例を挙げます。それぞれ異なる側面に焦点を当てています。

* **マクロ経済要因に焦点を当てた例:**  `2025年2月の金融市場動向：インフレ率、金利、原油価格の影響分析`

* **特定の市場セクターに焦点を当てた例:** `2025年2月の金融市場動向：テクノロジー株市場の動向と今後の展望`  または `2025年2月の金融市場動向：新興市場国債の動向とリスク要因`

* **地域に焦点を当てた例:** `2025年2月の金融市場動向：欧州株式市場の動向と米国市場との相関関係` または `2025年2月の金融市場動向：アジア太平洋地域の金融市場の動向と地政学的リスクの影響`

* **特定のイベントへの影響に焦点を当てた例:** `2025年2月の金融市場動向：最新の利上げ発表が株式市場に与えた影響分析` または  `2025年2月の金融市場動向：主要中央銀行の金融政策変更が為替レートに与えた影響`

* **予測モデルを含めた例:** `2025年2月の金融市場動向：時系列分析を用いた今後の市場予測とリスク評価`


これらの例は、元のクエリ「最近の金融市場動向」をより具体的な情報要求に拡張しています。  どの拡張が最適かは、ユーザーの具体的な興味や分析の目的によって異なります。



'いくつかの具体的に拡張したクエリ例を挙げます。それぞれ異なる側面に焦点を当てています。\n\n* **マクロ経済要因に焦点を当てた例:**  `2025年2月の金融市場動向：インフレ率、金利、原油価格の影響分析`\n\n* **特定の市場セクターに焦点を当てた例:** `2025年2月の金融市場動向：テクノロジー株市場の動向と今後の展望`  または `2025年2月の金融市場動向：新興市場国債の動向とリスク要因`\n\n* **地域に焦点を当てた例:** `2025年2月の金融市場動向：欧州株式市場の動向と米国市場との相関関係` または `2025年2月の金融市場動向：アジア太平洋地域の金融市場の動向と地政学的リスクの影響`\n\n* **特定のイベントへの影響に焦点を当てた例:** `2025年2月の金融市場動向：最新の利上げ発表が株式市場に与えた影響分析` または  `2025年2月の金融市場動向：主要中央銀行の金融政策変更が為替レートに与えた影響`\n\n* **予測モデルを含めた例:** `2025年2月の金融市場動向：時系列分析を用いた今後の市場予測とリスク評価`\n\n\nこれらの例は、元のクエリ「最近の金融市場動向」をより具体的な情報要求に拡張しています。  どの拡張が最適かは、ユーザーの具体的な興味や分析の目的によって異なります。'

In [127]:
query_embedding = get_gemini_embedding(query_text)  # 拡張せずそのまま

In [128]:
test_query_embedding = get_gemini_embedding("金融市場")
test_results = collections["daily"].query(
    query_embeddings=[test_query_embedding],
    n_results=5,
    include=["documents", "metadatas", "distances"]
)
print(f"🔍 テストクエリ検索結果: {test_results}")


🔍 テストクエリ検索結果: {'ids': [['d68bad8a-19a2-4b27-b498-34fc5065f4b0', '44118f1a-a0ec-41e7-9ea9-30c0c57854cb', '445cff83-d8b0-47d4-ac62-98d5b8b6562f', '46687a48-7f53-4c33-ae65-217d5cd941ca', '012c83bf-b44d-4b2f-bf4a-bb6508760dbd']], 'distances': [[0.6512641310691833, 0.7175018787384033, 0.7827810645103455, 0.8001096844673157, 0.8119246363639832]], 'embeddings': None, 'metadatas': [[{'category': 'daily', 'chunk_index': 6, 'source': 'input/data-rag/daily250220.pdf'}, {'category': 'daily', 'chunk_index': 11, 'source': 'input/data-rag/daily250219.pdf'}, {'category': 'daily', 'chunk_index': 7, 'source': 'input/data-rag/daily250220.pdf'}, {'category': 'daily', 'chunk_index': 14, 'source': 'input/data-rag/daily250221.pdf'}, {'category': 'daily', 'chunk_index': 5, 'source': 'input/data-rag/daily250221.pdf'}]], 'documents': [['為替（対円） （単位：円） 2月19日 2月18日 前日比% 懸念から、量的引き締めの減速や一時停止の検討の必\n米ドル 151.47 152.06 ▲0.39 要性が言及されたことが好感され、相場を下支え。\nユーロ 157.88 158.84 ▲0.60 ●欧州株は大幅下落。トランプ米大統領が、4月にも輸\n英ポンド 190.65 191.81 ▲